<h1>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

importing libraries

In [2]:
import pandas as pd
import numpy as np
from urllib import request
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
from bs4 import BeautifulSoup
print('Libraries imported')

Requirement not upgraded as not directly required: beautifulsoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: lxml in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: html5lib in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: webencodings in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: setuptools>=18.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Libraries imported


<b>Loading webpage from Wikipedia

In [3]:
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [4]:
page = request.Request(wiki)
Wiki = request.urlopen(page).read().decode('utf-8')

<b>Parsing webpage scraping table

In [5]:
soup = BeautifulSoup(Wiki, 'lxml')

In [6]:
match = soup.find('div', class_='mw-parser-output')

In [7]:
match2  = match.table.tbody

<b>Taking the Column names

In [8]:
col = match2.find_all('th')

In [9]:
col

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
 </th>]

In [10]:
column = []
for i,j in enumerate(col): column.append(col[i].text.strip())

In [11]:
column

['Postcode', 'Borough', 'Neighbourhood']

In [12]:
df_toronto = pd.DataFrame(columns = column)

<b> Filling up the data

In [13]:
data = match2.find_all('td')

In [14]:
borough = None
neigh = None
for i in range(int(len(data)/3)):
    k = i*3
    if not (data[k+1].text) == 'Not assigned': 
        borough = str(data[k+1].text)
    if not (data[k+2].text.strip()) == 'Not assigned': 
        neigh = str(data[k+2].text.strip())
    df_toronto.loc[i] = pd.Series({column[0]:data[k].text, column[1]:borough, column[2]:neigh})

<b>Dropping not assigned boroughs

In [15]:
df_toronto.dropna(inplace=True)

In [16]:
df_toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


<b>Merging same Post Code

In [17]:
df_t1 = df_toronto.groupby(df_toronto['Postcode'])['Neighbourhood'].apply(', '.join).reset_index()
df_t1.drop('Postcode', axis=1, inplace=True)

In [18]:
df_t2 = df_toronto.groupby('Postcode').agg({'Borough':'first'}).reset_index()

In [19]:
df_T = pd.concat([df_t2, df_t1], axis=1)

<h2>Results of part 1

In [20]:
df_T.shape

(178, 3)

In [21]:
df_T

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<H2> Assignment part 2

<b>Installing geocoder

In [22]:
!pip install geocoder
import geocoder # import geocoder

    100% |████████████████████████████████| 102kB 7.3MB/s ta 0:00:01
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda

<b> Verifiing if its work

In [23]:
# initialize your variable to None
lat_lng_coords = None
postal_code = 'M6N'
counter = 0
# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    counter += 1  
    if counter >= 200:
        break
    
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(postal_code, latitude, longitude, counter)

TypeError: 'NoneType' object is not subscriptable

<b> Didnt work, so let`s use the csv file.

<b> Reading the CSV into Pandas DataFrame

In [29]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
df_geo.shape

(103, 3)

<b>Joining the first DF with the second

In [31]:
print(df_geo.shape, df_T.shape)

(103, 3) (178, 3)


In [32]:
df_Toronto = df_T.join(df_geo.set_index('Postal Code'), on='Postcode')

<h2>Results of part 2

In [33]:
df_Toronto.dropna(inplace=True)
df_Toronto.shape

(103, 5)

In [34]:
df_Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h2>Assignment part 3

#### Define Foursquare Credentials and Version

In [35]:
# The code was removed by Watson Studio for sharing.

In [40]:
!pip install folium
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

    100% |████████████████████████████████| 92kB 4.7MB/s ta 0:00:011
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/

#### Use geopy library to get the latitude and longitude values of Toronto.

In [41]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of New York with neighborhoods superimposed on top.

In [62]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Neighbourhood, Postcode in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Neighbourhood'], df_Toronto['Postcode']):
    label = '{}, {}'.format(Postcode, Neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
        
map_toronto

#### Create a function to make the process to all the neighborhoods in Toronto

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    import requests
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Creating the venues dataframe
#### Distinguish by Postal Code

In [46]:
_venues = getNearbyVenues(names=df_Toronto['Postcode'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


In [47]:
_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M1B,1,1,1,1,1,1
M1C,3,3,3,3,3,3
M1E,6,6,6,6,6,6
M1G,3,3,3,3,3,3
M1H,7,7,7,7,7,7
M1J,2,2,2,2,2,2
M1K,7,7,7,7,7,7
M1L,10,10,10,10,10,10
M1M,2,2,2,2,2,2


## Analyze Each Neighborhood
### One hot encoding

In [48]:
# one hot encoding
_onehot = pd.get_dummies(_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
_onehot['Neighborhood'] = _venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [_onehot.columns[-1]] + list(_onehot.columns[:-1])
_onehot = _onehot[fixed_columns]

_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
_onehot.shape

(2229, 272)

In [50]:
_grouped = _onehot.groupby('Neighborhood').mean().reset_index()
_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,M1B,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
1,M1C,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
2,M1E,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
3,M1G,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
4,M1H,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
5,M1J,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
6,M1K,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.142857,0.00,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
7,M1L,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
8,M1M,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
9,M1N,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000


#### Taking the most common venues for each postal code

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
_venues_sorted = pd.DataFrame(columns=columns)
_venues_sorted['Postcode'] = _grouped['Neighborhood']

for ind in np.arange(_grouped.shape[0]):
    _venues_sorted.iloc[ind, 1:] = return_most_common_venues(_grouped.iloc[ind, :], num_top_venues)

_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Dessert Shop
1,M1C,Bar,Moving Target,Construction & Landscaping,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
2,M1E,Rental Car Location,Electronics Store,Pizza Place,Breakfast Spot,Medical Center,Mexican Restaurant,Women's Store,Doner Restaurant,Diner,Discount Store
3,M1G,Coffee Shop,Korean Restaurant,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store
4,M1H,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant,Bakery,Caribbean Restaurant,Bank,Athletics & Sports,Donut Shop,Dog Run,Doner Restaurant


#### Training the KNN algorithm to Cluster the dataset

In [53]:
# set number of clusters
kclusters = 5

_grouped_clustering = _grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 2, 2, 2, 2, 2, 0, 2], dtype=int32)

In [54]:
kmeans.labels_

array([0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 3, 0, 2, 2, 2, 2, 2, 3, 0,
       3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 0, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2,
       2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 3, 1, 2, 2,
       0, 0, 1, 2, 0, 3, 0, 2], dtype=int32)

#### Merging the results in a new dataset

In [74]:
#_venues_sorted.drop('Cluster Labels', axis = 1, inplace=True)

In [55]:
# add clustering labels
_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

_merged = df_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
_merged = _merged.join(_venues_sorted.set_index('Postcode'), on='Postcode')

_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Dessert Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2.0,Bar,Moving Target,Construction & Landscaping,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Rental Car Location,Electronics Store,Pizza Place,Breakfast Spot,Medical Center,Mexican Restaurant,Women's Store,Doner Restaurant,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Hakka Restaurant,Fried Chicken Joint,Thai Restaurant,Bakery,Caribbean Restaurant,Bank,Athletics & Sports,Donut Shop,Dog Run,Doner Restaurant


In [56]:
_merged.dropna(inplace=True)

## Results of part 3

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(_merged['Latitude'], _merged['Longitude'], _merged['Neighbourhood'], _merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters